In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime as dt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [28]:
new= pd.read_csv('newRes90_clean.csv', index_col = 0)
sec = pd.read_csv('secRes90_clean.csv', index_col = 0)
coord = pd.read_csv('coordiantes.csv', index_col = 0)
with open('inflation.json', 'r') as f:
    inf = json.load(f)

In [29]:
(new.dtypes != 'float64').sum()

0

In [30]:
new.columns = pd.to_datetime(new.columns, format='%Y%m').date
sec.columns = pd.to_datetime(sec.columns, format = '%Y%m').date

In [31]:
#sec = sec.join(coord)
#new = new.join(coord)

In [32]:
rw2 = widgets.RadioButtons(options = ['English', '中文'], value = 'English')

display(rw2)

RadioButtons(options=('English', '中文'), value='English')

In [33]:
if rw2.value == 'English':
    texts = {'w':{'pw':'Price',
                  'rw': ['Newly built','Second hand'],
                  'dw':'Date range'}
            }
                
else:
    texts = {'w':{'pw':'价钱',
                  'rw':['新建','用过的'],
                  'dw':'日期范围'}
            }

In [34]:
pw = widgets.IntSlider(description = texts['w']['pw'])
pw

IntSlider(value=0, description='Price')

In [35]:
if rw2.value == 'English':
    cw = widgets.Dropdown(
        options=new.index,
        description='City:',
        disabled=False,
        description_tooltip = 'Select a city',
    )
else: 
    cw = widgets.Dropdown(
        options=new['chinese_name'],
        description='市:',
        disabled=False,
    )
display(cw)

Dropdown(description='City:', description_tooltip='Select a city', options=('Anqing', 'Baotou', 'Beihai', 'Bei…

In [36]:
rw = widgets.RadioButtons(
    options= texts['w']['rw'],
     value= texts['w']['rw'][0],
    disabled=False,
)
display(rw)
rw.value

RadioButtons(options=('Newly built', 'Second hand'), value='Newly built')

'Newly built'

In [37]:
[i.strftime('%Y-%m') for i in new.columns[:69]][-1]

'2020-04'

In [38]:
dw = widgets.SelectionRangeSlider(
    options= [i.strftime('%Y-%m') for i in new.columns[:-4]],
    index=(0,len(new.columns[:-5])),
    description= texts['w']['dw'],
    value = ([i.strftime('%Y-%m') for i in new.columns[:1]][0],[i.strftime('%Y-%m') for i in new.columns[:69]][-1]),
    disabled=False
)
display(dw)
print(dw.value)


TraitError: Invalid selection: value not found

In [39]:
x = [dt.datetime.strptime(i, '%Y-%m').date() for i in dw.value]
print(x)

[datetime.date(2014, 7, 1), datetime.date(2020, 4, 1)]


In [40]:
if rw.value in ['Newly built', '新建']:
    d = new.copy()
else: d = sec.copy()

if rw2.value == 'English':
    df = d.loc[cw.value,x[0]:x[1]]
else:
    df = d.loc[d['chinese_name'] == cw.value].loc[:,x[0]:x[1]].join(coord['chinese_name'])
    
df

2014-07-01     99.7
2014-08-01     99.1
2014-09-01     98.5
2014-10-01     98.9
2014-11-01     99.3
              ...  
2019-12-01     99.4
2020-01-01    100.8
2020-02-01     99.9
2020-03-01    100.0
2020-04-01     99.5
Name: Anqing, Length: 69, dtype: float64

In [43]:
from ipywidgets import interactive
import matplotlib.font_manager as mfm
font_path = "/usr/share/fonts/truetype/NotoSansSC-Regular.otf"
font_path = "/usr/share/fonts/truetype/DroidSansFallbackFull.ttf"
font_path = "/usr/share/fonts/truetype/NotoSansTC-Bold.otf"
prop = mfm.FontProperties(fname=font_path)
plt.rcParams['font.family']= 'DejaVu Sans'

def _cdef(Date,City, Price):
    c = City       
    p = Price
    t = Date
    

    fig = plt.figure(figsize = (15,4))
    ax = fig.add_subplot(1,1,1)
    for i, l  in zip([new,sec],['new', '2nd hand']):
        i = i.loc[c, t:]
        
        plt.plot(i.index, i.values, label= c+' '+l)
        plt.fill_between(i.index, i.values,
            100, # the price of reference, buying price eg. exact No. also good
            where = (i.values > 100), #shows only when higher then the buying price
            facecolor = 'g',
            alpha = 0.5
            )
        plt.fill_between(i.index, i.values,
            100, # the price of reference, buying price eg. exact No. also good
            where = (i.values <= 100), #shows only when higher then the buying price
            facecolor = 'r',
            alpha = 0.5
            )

    plt.xticks(rotation = 45)
    plt.legend(loc = (0.8,0.85))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.axhline(100,color = 'red', linewidth = 1)
    plt.suptitle('Property price index change between {} and today'.format(t), fontsize = 16)
    plt.title('{}至今天之间的房地产价格变化'.format(t), fontproperties=prop, fontsize = 14)


    plt.show()

    #price calculation
    p1 = p
    if d == 'new':
        plist = new.loc[c,t:]
    else:
        plist = sec.loc[c,t:]
    for i in plist:
        p1 *=(float(i)/100)
    pp1 = p1/p*100


    print('The price increase in the last {} month is {:.2f} %, value {:0.0f}'.format(len(plist), pp1, p1))
    print('最近{}个月的价格上涨值为{:.2f} %, RMB {:0.0f}'.format(len(plist), pp1, p1))

    return



#t = input('date 201810: ')
d = input('Property type: ')
c = interactive(_cdef,Date = sec.columns, City = sec.index, Price = (200000,1500000,50000))

Property type: 


In [44]:
c

interactive(children=(Dropdown(description='Date', options=(datetime.date(2014, 7, 1), datetime.date(2014, 8, …